In [1]:
import json
import re
from pprint import pprint
from collections import Counter
import numpy as np
#import stanza
#stanza.download('et')

In [2]:
from collections import defaultdict

In [6]:
rng = np.random.default_rng()
rand_ints = rng.integers(0, 572, size=5)

In [7]:
rand_ints

array([232, 280,   4, 402, 201], dtype=int64)

In [11]:
label_counts = Counter()
total_words = 0
doc_count = 0
doc_lengths = []

with open('data/estner.json') as corpus:
    i = 0
    for doc in corpus:
        estnltk_dict = json.loads(doc.strip())
        doc_count += 1
        doc_length = 0
        for word in estnltk_dict.get('words', []):
            total_words += 1
            label_counts[word['label']] += 1
            doc_length += 1
        doc_lengths.append(doc_length)
        if i in rand_ints:
            pprint(estnltk_dict)
        i += 1

{'paragraphs': [{'end': 1106, 'start': 0}],
 'sentences': [{'end': 40, 'start': 0},
               {'end': 76, 'start': 41},
               {'end': 133, 'start': 77},
               {'end': 204, 'start': 134},
               {'end': 277, 'start': 205},
               {'end': 480, 'start': 278},
               {'end': 587, 'start': 481},
               {'end': 639, 'start': 588},
               {'end': 714, 'start': 640},
               {'end': 793, 'start': 715},
               {'end': 854, 'start': 794},
               {'end': 886, 'start': 855},
               {'end': 983, 'start': 887},
               {'end': 1021, 'start': 984},
               {'end': 1081, 'start': 1022},
               {'end': 1103, 'start': 1082},
               {'end': 1106, 'start': 1104}],
 'text': 'Verine kaklus lõpetas Oksmaa kontserdi .\n'
         '„ See kõik käis nagu action-filmis! ” kirjeldab Arnold Oksmaa SL '
         'Õhtulehele baarilöömingut .\n'
         '„ Kes ette jäi , see sai , ” meenutab tem

           {'end': 1158, 'label': 'O', 'start': 1150, 'text': 'täitvate'},
           {'end': 1167, 'label': 'O', 'start': 1159, 'text': 'inimeste'},
           {'end': 1186,
            'label': 'O',
            'start': 1168,
            'text': 'ebakompetentsusest'},
           {'end': 1188, 'label': 'O', 'start': 1187, 'text': '.'},
           {'end': 1192, 'label': 'O', 'start': 1189, 'text': 'Ent'},
           {'end': 1206, 'label': 'O', 'start': 1193, 'text': 'tööülesannete'},
           {'end': 1215, 'label': 'O', 'start': 1207, 'text': 'jagamine'},
           {'end': 1222, 'label': 'O', 'start': 1216, 'text': 'teiste'},
           {'end': 1233, 'label': 'O', 'start': 1223, 'text': 'ministrite'},
           {'end': 1239, 'label': 'O', 'start': 1234, 'text': 'vahel'},
           {'end': 1245, 'label': 'O', 'start': 1240, 'text': 'oleks'},
           {'end': 1253, 'label': 'O', 'start': 1246, 'text': 'mõeldav'},
           {'end': 1256, 'label': 'O', 'start': 1254, 'text': 'ja'},

Viie juhusliku artikli analüüsist selgub, et treeningkorpus sisaldab vigu ja tuleks tervenisti üle kontrollida. Näiteks on mõnel juhul kaks järjestikust, ent eraldiseisvat nimeüksust liidetud üheks, või on lihtsalt üksikule nimeüksusele pandud vale märgend (nt B-LOC, mitte B-PER).

In [13]:
prev_label = ''
with open('data/estner.json') as corpus:
    for doc in corpus:
        estnltk_dict = json.loads(doc.strip())
        for word in estnltk_dict.get('words', []):
            current_label = word['label']
            if (prev_label.startswith('B-') and current_label.startswith('I-')
                and prev_label[-3:] != current_label[-3:]):
                print('ERROR:', prev_label, 'followed by', current_label)
            prev_label = current_label

Siiski ei leidu korpuses ühtki sellist juhtu, kus üksteisele järgnevad üht tüüpi algus ja teist tüüpi sisu (nt B-PER ja seejärel I-LOC).

In [14]:
prev_label = ''
with open('data/estner.json') as corpus:
    for doc in corpus:
        estnltk_dict = json.loads(doc.strip())
        for word in estnltk_dict.get('words', []):
            current_label = word['label']
            if prev_label == 'O' and current_label.startswith('I-'):
                print('ERROR:', prev_label, 'followed by', current_label)
            prev_label = current_label

Samuti ei leidu juhtu, kus O-tüüpi sõnale järgneb nimeolemi sisu ehk nt I-PER.

In [62]:
def with_context(word_idx, word_dict):
    word_count = len(word_dict)
    context_start = (0 if word_idx < 4 else word_idx-4)
    context_end = (word_count if word_count-word_idx < 4 else word_idx+5)
    return [word['text'] for word in word_dict[context_start:context_end]]

In [13]:
punctuation = re.compile('\.|\,$')
with open('data/estner.json') as corpus:
    for doc in corpus:
        estnltk_dict = json.loads(doc.strip())
        word_dict = estnltk_dict.get('words', [])
        for idx, word in enumerate(word_dict):
            current_label = word['label']
            current_word = word['text']
            if current_label != 'O' and re.match(punctuation, current_word):
                print('ERROR:', current_word, 'labelled as', current_label)
                print(with_context(idx, word_dict))

ERROR: . labelled as I-PER
['kaitsejõudude', 'leitnant', 'Tiit', 'Uustalu', '.', '”', 'Uustalu', 'väitel', 'veeti']
ERROR: . labelled as I-ORG
['Zimbabwesse', ',', 'teatab', 'CNN', '.', 'Uus-Meremaa', 'välisministri', 'sõnul', 'on']
ERROR: . labelled as I-ORG
['sõnas', 'Kukumägi', 'ajakirjas', 'Teater.Muusika.Kino', '.', 'viimases', 'numbris', '.', 'Samas']
ERROR: . labelled as I-LOC
['ilmselt', 'alles', 'maandumisel', 'Tallinnas', '.', '”', 'Lennuki', 'teliku', 'rehvid']
ERROR: . labelled as I-PER
['a', '.', 'Sirje', 'Jõgiste', '.', '”', '.', 'Seega', 'minister']
ERROR: . labelled as I-PER
['Ariko', 'uuringujuht', 'Kalev', 'Petti', '.', '14', 'protsenti', 'vastanutest', 'tahab']
ERROR: . labelled as I-ORG
['pühkida', ',', 'teatab', 'BBC', '.', '“', 'Kas', 'te', 'saate']
ERROR: . labelled as I-LOC
[',', 'Iisrael', ',', 'Venemaa', '.', '”', 'Olmerti', 'pressiesindaja', 'ütles']
ERROR: . labelled as I-LOC
['sama', 'teeb', 'ka', 'USA', '.', 'Boikoti', 'jätkamise', 'põhjuseks', 'on']
ERROR

In [ ]:
with open('data/estner.json') as corpus:
    doc_idx = 0
    for doc in corpus:
        estnltk_dict = json.loads(doc.strip())
        #for word in estnltk_dict.get('words', []):
        #    pass
        input('Press ENTER for next document')
        print('Document no.:', doc_idx)
        for idx, word in enumerate(estnltk_dict.get('words', [])):
            print(idx, word)
        doc_idx += 1

Press ENTER for next document
Document no.: 0
0 {'label': 'B-LOC', 'start': 0, 'end': 8, 'text': 'Tallinna'}
1 {'label': 'O', 'start': 9, 'end': 18, 'text': 'õhusaaste'}
2 {'label': 'O', 'start': 19, 'end': 27, 'text': 'suureneb'}
3 {'label': 'O', 'start': 28, 'end': 29, 'text': '.'}
4 {'label': 'O', 'start': 30, 'end': 35, 'text': 'Kuigi'}
5 {'label': 'O', 'start': 36, 'end': 44, 'text': 'möödunud'}
6 {'label': 'O', 'start': 45, 'end': 51, 'text': 'nädala'}
7 {'label': 'O', 'start': 52, 'end': 56, 'text': 'ühel'}
8 {'label': 'O', 'start': 57, 'end': 60, 'text': 'ööl'}
9 {'label': 'O', 'start': 61, 'end': 71, 'text': 'ootamatult'}
10 {'label': 'O', 'start': 72, 'end': 79, 'text': 'kõrgeks'}
11 {'label': 'O', 'start': 80, 'end': 87, 'text': 'tõusnud'}
12 {'label': 'O', 'start': 88, 'end': 97, 'text': 'õhusaaste'}
13 {'label': 'O', 'start': 98, 'end': 101, 'text': 'oli'}
14 {'label': 'O', 'start': 102, 'end': 110, 'text': 'pealinna'}
15 {'label': 'O', 'start': 111, 'end': 116, 'text': 'j

Press ENTER for next document
Document no.: 1
0 {'label': 'O', 'start': 0, 'end': 9, 'text': 'Politseid'}
1 {'label': 'O', 'start': 10, 'end': 18, 'text': 'vihastab'}
2 {'label': 'O', 'start': 19, 'end': 29, 'text': 'pätiplaani'}
3 {'label': 'O', 'start': 30, 'end': 44, 'text': 'avalikustamine'}
4 {'label': 'O', 'start': 45, 'end': 46, 'text': '.'}
5 {'label': 'O', 'start': 47, 'end': 56, 'text': 'Politseid'}
6 {'label': 'O', 'start': 57, 'end': 65, 'text': 'pahandab'}
7 {'label': 'O', 'start': 66, 'end': 67, 'text': ','}
8 {'label': 'O', 'start': 68, 'end': 70, 'text': 'et'}
9 {'label': 'O', 'start': 71, 'end': 91, 'text': 'justiitsministeerium'}
10 {'label': 'O', 'start': 92, 'end': 103, 'text': 'avalikustas'}
11 {'label': 'O', 'start': 104, 'end': 109, 'text': 'nende'}
12 {'label': 'O', 'start': 110, 'end': 125, 'text': 'asutusesiseseks'}
13 {'label': 'O', 'start': 126, 'end': 138, 'text': 'kasutamiseks'}
14 {'label': 'O', 'start': 139, 'end': 146, 'text': 'mõeldud'}
15 {'label': 'O

Press ENTER for next document
Document no.: 2
0 {'label': 'O', 'start': 0, 'end': 4, 'text': 'Mees'}
1 {'label': 'O', 'start': 5, 'end': 9, 'text': 'ajas'}
2 {'label': 'O', 'start': 10, 'end': 23, 'text': 'ülekäigurajal'}
3 {'label': 'O', 'start': 24, 'end': 29, 'text': 'naise'}
4 {'label': 'O', 'start': 30, 'end': 34, 'text': 'alla'}
5 {'label': 'O', 'start': 35, 'end': 36, 'text': '.'}
6 {'label': 'O', 'start': 37, 'end': 47, 'text': 'Keskealine'}
7 {'label': 'O', 'start': 48, 'end': 53, 'text': 'naine'}
8 {'label': 'O', 'start': 54, 'end': 57, 'text': 'jäi'}
9 {'label': 'O', 'start': 58, 'end': 67, 'text': 'laupäeval'}
10 {'label': 'B-LOC', 'start': 68, 'end': 77, 'text': 'Tallinnas'}
11 {'label': 'O', 'start': 78, 'end': 90, 'text': 'reguleeritud'}
12 {'label': 'O', 'start': 91, 'end': 104, 'text': 'ülekäigurajal'}
13 {'label': 'O', 'start': 105, 'end': 109, 'text': 'auto'}
14 {'label': 'O', 'start': 110, 'end': 114, 'text': 'alla'}
15 {'label': 'O', 'start': 115, 'end': 119, 'text

Press ENTER for next document
Document no.: 3
0 {'label': 'B-LOC', 'start': 0, 'end': 4, 'text': 'Vene'}
1 {'label': 'O', 'start': 5, 'end': 9, 'text': 'väed'}
2 {'label': 'O', 'start': 10, 'end': 16, 'text': 'võivad'}
3 {'label': 'B-LOC', 'start': 17, 'end': 26, 'text': 'Gruusiast'}
4 {'label': 'O', 'start': 27, 'end': 34, 'text': 'lahkuda'}
5 {'label': 'O', 'start': 35, 'end': 36, 'text': '.'}
6 {'label': 'B-LOC', 'start': 37, 'end': 44, 'text': 'Venemaa'}
7 {'label': 'O', 'start': 45, 'end': 49, 'text': 'võib'}
8 {'label': 'O', 'start': 50, 'end': 58, 'text': 'alustada'}
9 {'label': 'O', 'start': 59, 'end': 65, 'text': 'vägede'}
10 {'label': 'O', 'start': 66, 'end': 78, 'text': 'väljaviimist'}
11 {'label': 'B-LOC', 'start': 79, 'end': 88, 'text': 'Gruusiast'}
12 {'label': 'O', 'start': 89, 'end': 92, 'text': 'sel'}
13 {'label': 'O', 'start': 93, 'end': 99, 'text': 'aastal'}
14 {'label': 'O', 'start': 100, 'end': 101, 'text': ','}
15 {'label': 'O', 'start': 102, 'end': 108, 'text': '

Press ENTER for next document
Document no.: 4
0 {'label': 'O', 'start': 0, 'end': 6, 'text': 'Verine'}
1 {'label': 'O', 'start': 7, 'end': 13, 'text': 'kaklus'}
2 {'label': 'O', 'start': 14, 'end': 21, 'text': 'lõpetas'}
3 {'label': 'B-LOC', 'start': 22, 'end': 28, 'text': 'Oksmaa'}
4 {'label': 'O', 'start': 29, 'end': 38, 'text': 'kontserdi'}
5 {'label': 'O', 'start': 39, 'end': 40, 'text': '.'}
6 {'label': 'O', 'start': 41, 'end': 42, 'text': '„'}
7 {'label': 'O', 'start': 43, 'end': 46, 'text': 'See'}
8 {'label': 'O', 'start': 47, 'end': 51, 'text': 'kõik'}
9 {'label': 'O', 'start': 52, 'end': 56, 'text': 'käis'}
10 {'label': 'O', 'start': 57, 'end': 61, 'text': 'nagu'}
11 {'label': 'O', 'start': 62, 'end': 75, 'text': 'action-filmis'}
12 {'label': 'O', 'start': 75, 'end': 76, 'text': '!'}
13 {'label': 'O', 'start': 77, 'end': 78, 'text': '”'}
14 {'label': 'O', 'start': 79, 'end': 88, 'text': 'kirjeldab'}
15 {'label': 'B-PER', 'start': 89, 'end': 95, 'text': 'Arnold'}
16 {'label': '

...

Press ENTER for next document
Document no.: 14
0 {'label': 'B-ORG', 'start': 0, 'end': 12, 'text': 'Edelaraudtee'}
1 {'label': 'O', 'start': 13, 'end': 18, 'text': 'tahab'}
2 {'label': 'O', 'start': 19, 'end': 30, 'text': 'piletihinda'}
3 {'label': 'O', 'start': 31, 'end': 36, 'text': 'tõsta'}
4 {'label': 'O', 'start': 37, 'end': 38, 'text': '.'}
5 {'label': 'B-ORG', 'start': 39, 'end': 51, 'text': 'Edelaraudtee'}
6 {'label': 'O', 'start': 52, 'end': 60, 'text': 'otsustab'}
7 {'label': 'O', 'start': 61, 'end': 64, 'text': 'sel'}
8 {'label': 'O', 'start': 65, 'end': 72, 'text': 'nädalal'}
9 {'label': 'O', 'start': 73, 'end': 74, 'text': ','}
10 {'label': 'O', 'start': 75, 'end': 78, 'text': 'kas'}
11 {'label': 'O', 'start': 79, 'end': 84, 'text': 'tõsta'}
12 {'label': 'O', 'start': 85, 'end': 96, 'text': 'rongipileti'}
13 {'label': 'O', 'start': 97, 'end': 102, 'text': 'hinda'}
14 {'label': 'O', 'start': 103, 'end': 107, 'text': 'kuni'}
15 {'label': 'O', 'start': 108, 'end': 117, 'text'

Press ENTER for next document
Document no.: 15
0 {'label': 'O', 'start': 0, 'end': 6, 'text': 'Tabati'}
1 {'label': 'B-LOC', 'start': 7, 'end': 14, 'text': 'Londoni'}
2 {'label': 'O', 'start': 15, 'end': 24, 'text': 'ründajate'}
3 {'label': 'O', 'start': 25, 'end': 42, 'text': 'dokumendivõltsija'}
4 {'label': 'O', 'start': 43, 'end': 44, 'text': '.'}
5 {'label': 'B-LOC', 'start': 45, 'end': 48, 'text': 'Tai'}
6 {'label': 'O', 'start': 49, 'end': 57, 'text': 'politsei'}
7 {'label': 'O', 'start': 58, 'end': 66, 'text': 'vahistas'}
8 {'label': 'B-LOC', 'start': 67, 'end': 75, 'text': 'Alžeeria'}
9 {'label': 'O', 'start': 76, 'end': 84, 'text': 'päritolu'}
10 {'label': 'O', 'start': 85, 'end': 89, 'text': 'mehe'}
11 {'label': 'O', 'start': 90, 'end': 91, 'text': ','}
12 {'label': 'O', 'start': 92, 'end': 96, 'text': 'keda'}
13 {'label': 'O', 'start': 97, 'end': 111, 'text': 'kahtlustatakse'}
14 {'label': 'O', 'start': 112, 'end': 120, 'text': 'seotuses'}
15 {'label': 'O', 'start': 121, 'en

Press ENTER for next document
Document no.: 17
0 {'label': 'B-ORG', 'start': 0, 'end': 4, 'text': 'NATO'}
1 {'label': 'O', 'start': 5, 'end': 17, 'text': 'laskemoonast'}
2 {'label': 'O', 'start': 18, 'end': 23, 'text': 'leiti'}
3 {'label': 'O', 'start': 24, 'end': 34, 'text': 'rikastatud'}
4 {'label': 'O', 'start': 35, 'end': 41, 'text': 'uraani'}
5 {'label': 'O', 'start': 42, 'end': 43, 'text': '.'}
6 {'label': 'B-ORG', 'start': 44, 'end': 47, 'text': 'ÜRO'}
7 {'label': 'I-ORG', 'start': 48, 'end': 65, 'text': 'Keskkonnaprogramm'}
8 {'label': 'O', 'start': 66, 'end': 67, 'text': '('}
9 {'label': 'B-ORG', 'start': 68, 'end': 72, 'text': 'UNEP'}
10 {'label': 'O', 'start': 73, 'end': 74, 'text': ')'}
11 {'label': 'O', 'start': 75, 'end': 80, 'text': 'ütles'}
12 {'label': 'O', 'start': 81, 'end': 92, 'text': 'kolmapäeval'}
13 {'label': 'O', 'start': 93, 'end': 94, 'text': ','}
14 {'label': 'O', 'start': 95, 'end': 97, 'text': 'et'}
15 {'label': 'B-LOC', 'start': 98, 'end': 105, 'text': 'Š

Press ENTER for next document
Document no.: 18
0 {'label': 'O', 'start': 0, 'end': 8, 'text': 'Eestlane'}
1 {'label': 'O', 'start': 9, 'end': 15, 'text': 'osales'}
2 {'label': 'B-LOC', 'start': 16, 'end': 26, 'text': 'Inglismaal'}
3 {'label': 'O', 'start': 27, 'end': 40, 'text': 'juveeliröövis'}
4 {'label': 'O', 'start': 41, 'end': 42, 'text': '.'}
5 {'label': 'O', 'start': 43, 'end': 47, 'text': 'Kolm'}
6 {'label': 'O', 'start': 48, 'end': 60, 'text': 'kurjategijat'}
7 {'label': 'O', 'start': 61, 'end': 69, 'text': 'röövisid'}
8 {'label': 'B-LOC', 'start': 70, 'end': 80, 'text': 'Inglismaal'}
9 {'label': 'O', 'start': 81, 'end': 91, 'text': 'juveeliäri'}
10 {'label': 'O', 'start': 92, 'end': 93, 'text': ','}
11 {'label': 'O', 'start': 94, 'end': 99, 'text': 'kinni'}
12 {'label': 'O', 'start': 100, 'end': 105, 'text': 'peeti'}
13 {'label': 'B-LOC', 'start': 106, 'end': 113, 'text': 'Eestist'}
14 {'label': 'O', 'start': 114, 'end': 119, 'text': 'pärit'}
15 {'label': 'O', 'start': 120, '

Press ENTER for next document
Document no.: 20
0 {'label': 'B-LOC', 'start': 0, 'end': 12, 'text': 'Kesk-Rootsis'}
1 {'label': 'B-LOC', 'start': 13, 'end': 23, 'text': 'Karlstadis'}
2 {'label': 'O', 'start': 24, 'end': 35, 'text': 'evakueeriti'}
3 {'label': 'O', 'start': 36, 'end': 48, 'text': 'lennujaamast'}
4 {'label': 'O', 'start': 49, 'end': 53, 'text': 'kõik'}
5 {'label': 'O', 'start': 54, 'end': 62, 'text': 'reisijad'}
6 {'label': 'O', 'start': 63, 'end': 65, 'text': 'ja'}
7 {'label': 'O', 'start': 66, 'end': 74, 'text': 'töötajad'}
8 {'label': 'O', 'start': 75, 'end': 76, 'text': ','}
9 {'label': 'O', 'start': 77, 'end': 81, 'text': 'sest'}
10 {'label': 'O', 'start': 82, 'end': 95, 'text': 'lennukompanii'}
11 {'label': 'B-ORG', 'start': 96, 'end': 99, 'text': 'SAS'}
12 {'label': 'O', 'start': 100, 'end': 107, 'text': 'töötaja'}
13 {'label': 'O', 'start': 108, 'end': 114, 'text': 'teatas'}
14 {'label': 'O', 'start': 115, 'end': 126, 'text': 'võimalikust'}
15 {'label': 'O', 'start

In [5]:
stanza_nlp = stanza.Pipeline(lang='et', processors='tokenize, pos', 
                             tokenize_pretokenized=True )

2020-04-25 11:17:42 INFO: Loading these models for language: et (Estonian):
| Processor | Package |
-----------------------
| tokenize  | edt     |
| pos       | edt     |

2020-04-25 11:17:42 INFO: Use device: cpu
2020-04-25 11:17:42 INFO: Loading: tokenize
2020-04-25 11:17:42 INFO: Loading: pos
2020-04-25 11:17:43 INFO: Done loading processors!


In [3]:
def estnltk_text_to_sentences(estnltk_dict):
    sentences = []
    sentence_starts = [sentence['start'] for sentence in estnltk_dict.get('sentences', [])]
    sentence_ends = [sentence['end'] for sentence in estnltk_dict.get('sentences', [])]
    sentence = []
    for word in estnltk_dict.get('words', []):
        start = word['start']
        end = word['end']
        if start in sentence_starts:
            sentence.append((word['text'], word['label']))
        elif end in sentence_ends:
            sentence.append((word['text'], word['label']))
            sentences.append(sentence)
            sentence = []
        else:
            sentence.append((word['text'], word['label']))
    # Don't forget the last sentence
    if sentence:
        sentences.append(sentence)
    return sentences 

In [88]:
possible_errors = 0
# Store possible correction cases here
correction_dict = defaultdict(list)
with open('data/estner_punct_cleaned.json') as corpus:
    text_idx = 0
    for line in corpus:
        estnltk_dict = json.loads(line.strip())
        
        if estnltk_dict:
            word_dict = estnltk_dict.get('words', [])

            estnltk_sentences = estnltk_text_to_sentences(estnltk_dict)

            stanza_doc = stanza_nlp(estnltk_sentences)
            stanza_words = [word for sentence in stanza_doc.sentences for word in sentence.words]

            # Align NER-labels and Stanza UPOS tags
            for idx, (estnltk_word, stanza_word) in enumerate(zip(word_dict, stanza_words)):
                estnltk_text = estnltk_word['text']
                stanza_text = stanza_word.text

                label = estnltk_word['label']
                upos = stanza_word.pos
        
                if (label == 'O' and upos == 'PROPN'):
            #(label != 'O' and upos in ('PUNCT', 'SYM', 'X')): 
            #or\
                    possible_errors += 1
                    print('POSSIBLE ERROR:')
                    print(estnltk_text, stanza_text, label, upos)
                    correction_dict[text_idx].append(idx)
        text_idx += 1

POSSIBLE ERROR:
Põhja Põhja O PROPN
POSSIBLE ERROR:
Estonia Estonia O PROPN
POSSIBLE ERROR:
Parvlaeva Parvlaeva O PROPN
POSSIBLE ERROR:
Estonia Estonia O PROPN
POSSIBLE ERROR:
Kesknädalas Kesknädalas O PROPN
POSSIBLE ERROR:
Parvlaeva Parvlaeva O PROPN
POSSIBLE ERROR:
Leitnant Leitnant O PROPN
POSSIBLE ERROR:
Estonia Estonia O PROPN
POSSIBLE ERROR:
Estonia Estonia O PROPN
POSSIBLE ERROR:
Gazeta.ru Gazeta.ru O PROPN
POSSIBLE ERROR:
Pronkssõduri Pronkssõduri O PROPN
POSSIBLE ERROR:
Purjel Purjel O PROPN
POSSIBLE ERROR:
Neoplan Neoplan O PROPN
POSSIBLE ERROR:
Reporteri Reporteri O PROPN
POSSIBLE ERROR:
Tallinn-2 Tallinn-2 O PROPN
POSSIBLE ERROR:
Spiezi Spiezi O PROPN
POSSIBLE ERROR:
AA AA O PROPN
POSSIBLE ERROR:
Tagasimaksel Tagasimaksel O PROPN
POSSIBLE ERROR:
Infokeskuse Infokeskuse O PROPN
POSSIBLE ERROR:
Iraani Iraani O PROPN
POSSIBLE ERROR:
Piduses Piduses O PROPN
POSSIBLE ERROR:
Palmisaarel Palmisaarel O PROPN
POSSIBLE ERROR:
American American O PROPN
POSSIBLE ERROR:
Beauty Beauty O 

POSSIBLE ERROR:
Vankumatu Vankumatu O PROPN
POSSIBLE ERROR:
Vabadus Vabadus O PROPN
POSSIBLE ERROR:
Force Force O PROPN
POSSIBLE ERROR:
F-35 F-35 O PROPN
POSSIBLE ERROR:
Joint Joint O PROPN
POSSIBLE ERROR:
Strike Strike O PROPN
POSSIBLE ERROR:
Fighter Fighter O PROPN
POSSIBLE ERROR:
Force Force O PROPN
POSSIBLE ERROR:
Sõjalise Sõjalise O PROPN
POSSIBLE ERROR:
Keskfraktsiooni Keskfraktsiooni O PROPN
POSSIBLE ERROR:
Ida Ida O PROPN
POSSIBLE ERROR:
Ida Ida O PROPN
POSSIBLE ERROR:
Kunst.ee Kunst.ee O PROPN
POSSIBLE ERROR:
Järvevana Järvevana O PROPN
POSSIBLE ERROR:
Kommersant Kommersant O PROPN
POSSIBLE ERROR:
Delfil Delfil O PROPN
POSSIBLE ERROR:
Julgeolekunõukogu Julgeolekunõukogu O PROPN
POSSIBLE ERROR:
Tagasivõtt Tagasivõtt O PROPN
POSSIBLE ERROR:
Superstaariks Superstaariks O PROPN
POSSIBLE ERROR:
VIDEO VIDEO O PROPN
POSSIBLE ERROR:
PUBLIK PUBLIK O PROPN
POSSIBLE ERROR:
Superstaarisaate Superstaarisaate O PROPN
POSSIBLE ERROR:
Superstaariks Superstaariks O PROPN
POSSIBLE ERROR:
Wonder

POSSIBLE ERROR:
Cobra Cobra O PROPN
POSSIBLE ERROR:
Osariiki Osariiki O PROPN
POSSIBLE ERROR:
Mercedes Mercedes O PROPN
POSSIBLE ERROR:
Benzi Benzi O PROPN
POSSIBLE ERROR:
Reede Reede O PROPN
POSSIBLE ERROR:
Noorteühingu Noorteühingu O PROPN
POSSIBLE ERROR:
Alama Alama O PROPN
POSSIBLE ERROR:
Universium Universium O PROPN
POSSIBLE ERROR:
Universium Universium O PROPN
POSSIBLE ERROR:
EurActiv.com EurActiv.com O PROPN
POSSIBLE ERROR:
Komisjoni Komisjoni O PROPN
POSSIBLE ERROR:
Annenbergi Annenbergi O PROPN
POSSIBLE ERROR:
Kiin Kiin O PROPN
POSSIBLE ERROR:
Valdoks Valdoks O PROPN
POSSIBLE ERROR:
Kuus Kuus O PROPN
POSSIBLE ERROR:
BMW BMW O PROPN
POSSIBLE ERROR:
Z8 Z8 O PROPN
POSSIBLE ERROR:
Kroonlinna Kroonlinna O PROPN
POSSIBLE ERROR:
Oti Oti O PROPN
POSSIBLE ERROR:
Ott Ott O PROPN
POSSIBLE ERROR:
Britt Britt O PROPN
POSSIBLE ERROR:
Muhulgas Muhulgas O PROPN
POSSIBLE ERROR:
Parlamendi Parlamendi O PROPN
POSSIBLE ERROR:
TT TT O PROPN
POSSIBLE ERROR:
Statistikaameti Statistikaameti O PROPN


UEFA UEFA O PROPN
POSSIBLE ERROR:
Profiratturid Profiratturid O PROPN
POSSIBLE ERROR:
Uusprofi Uusprofi O PROPN
POSSIBLE ERROR:
Väravavaht Väravavaht O PROPN
POSSIBLE ERROR:
Allah Allah O PROPN
POSSIBLE ERROR:
Nehv Nehv O PROPN
POSSIBLE ERROR:
Jork Jork O PROPN
POSSIBLE ERROR:
Allahiks Allahiks O PROPN
POSSIBLE ERROR:
Vene Vene O PROPN
POSSIBLE ERROR:
Nagano Nagano O PROPN
POSSIBLE ERROR:
MMile MMile O PROPN
POSSIBLE ERROR:
EestiÕigusjärgse EestiÕigusjärgse O PROPN
POSSIBLE ERROR:
Omandi Omandi O PROPN
POSSIBLE ERROR:
Ar Ar O PROPN
POSSIBLE ERROR:
Akadeemik Akadeemik O PROPN
POSSIBLE ERROR:
President President O PROPN
POSSIBLE ERROR:
Uue Uue O PROPN
POSSIBLE ERROR:
Telekomi Telekomi O PROPN
POSSIBLE ERROR:
Doonoreid Doonoreid O PROPN
POSSIBLE ERROR:
Atüüpi Atüüpi O PROPN
POSSIBLE ERROR:
Btüüpi Btüüpi O PROPN
POSSIBLE ERROR:
Sotsiaalministeerium Sotsiaalministeerium O PROPN
POSSIBLE ERROR:
PHARE PHARE O PROPN
POSSIBLE ERROR:
PHARE PHARE O PROPN
POSSIBLE ERROR:
PHARE PHARE O PROPN
POSSIB

POSSIBLE ERROR:
Volle Volle O PROPN
POSSIBLE ERROR:
Eesti Eesti O PROPN
POSSIBLE ERROR:
Naistennisistide Naistennisistide O PROPN
POSSIBLE ERROR:
WTA WTA O PROPN
POSSIBLE ERROR:
Austraalia Austraalia O PROPN
POSSIBLE ERROR:
MMil MMil O PROPN
POSSIBLE ERROR:
Eile Eile O PROPN
POSSIBLE ERROR:
Sevilla Sevilla O PROPN
POSSIBLE ERROR:
MMilt MMilt O PROPN
POSSIBLE ERROR:
Sydney Sydney O PROPN
POSSIBLE ERROR:
Suusakahevõistlejate Suusakahevõistlejate O PROPN
POSSIBLE ERROR:
Inglismaa Inglismaa O PROPN
POSSIBLE ERROR:
MONACO MONACO O PROPN
POSSIBLE ERROR:
MARATON MARATON O PROPN
POSSIBLE ERROR:
Monaco Monaco O PROPN
POSSIBLE ERROR:
Juhendaja Juhendaja O PROPN
POSSIBLE ERROR:
Rotterdami Rotterdami O PROPN
POSSIBLE ERROR:
Sevilla Sevilla O PROPN
POSSIBLE ERROR:
MM MM O PROPN
POSSIBLE ERROR:
Rotterdami Rotterdami O PROPN
POSSIBLE ERROR:
Vähim Vähim O PROPN
POSSIBLE ERROR:
NHL NHL O PROPN
POSSIBLE ERROR:
la la O PROPN
POSSIBLE ERROR:
la la O PROPN
POSSIBLE ERROR:
Mäesuusatamise Mäesuusatamise O PR

POSSIBLE ERROR:
Naeri Naeri O PROPN
POSSIBLE ERROR:
KAASSAAK KAASSAAK O PROPN
POSSIBLE ERROR:
Tungla Tungla O PROPN
POSSIBLE ERROR:
Imeerikas Imeerikas O PROPN
POSSIBLE ERROR:
Padakonnaoja Padakonnaoja O PROPN
POSSIBLE ERROR:
Rahvamats Rahvamats O PROPN
POSSIBLE ERROR:
Cook Cook O PROPN
POSSIBLE ERROR:
Priima Priima O PROPN
POSSIBLE ERROR:
Guinnessi Guinnessi O PROPN
POSSIBLE ERROR:
James James O PROPN
POSSIBLE ERROR:
Inglise Inglise O PROPN
POSSIBLE ERROR:
Sarjast Sarjast O PROPN
POSSIBLE ERROR:
Eesti Eesti O PROPN
POSSIBLE ERROR:
Raamat Raamat O PROPN
POSSIBLE ERROR:
Inglise Inglise O PROPN
POSSIBLE ERROR:
Inglise Inglise O PROPN
POSSIBLE ERROR:
Taani Taani O PROPN
POSSIBLE ERROR:
Sarjast Sarjast O PROPN
POSSIBLE ERROR:
Eesti Eesti O PROPN
POSSIBLE ERROR:
Mitmekülgse Mitmekülgse O PROPN
POSSIBLE ERROR:
JÄÄR JÄÄR O PROPN
POSSIBLE ERROR:
Jäär Jäär O PROPN
POSSIBLE ERROR:
Sõnn Sõnn O PROPN
POSSIBLE ERROR:
Sõnn Sõnn O PROPN
POSSIBLE ERROR:
Kaksikutel Kaksikutel O PROPN
POSSIBLE ERROR:
Vä

POSSIBLE ERROR:
Abba-projekt Abba-projekt O PROPN
POSSIBLE ERROR:
Müts Müts O PROPN
POSSIBLE ERROR:
Poeedi Poeedi O PROPN
POSSIBLE ERROR:
Gödel Gödel O PROPN
POSSIBLE ERROR:
Escher Escher O PROPN
POSSIBLE ERROR:
Bach Bach O PROPN
POSSIBLE ERROR:
Poetry Poetry O PROPN
POSSIBLE ERROR:
Motion Motion O PROPN
POSSIBLE ERROR:
From From O PROPN
POSSIBLE ERROR:
Nobeli Nobeli O PROPN
POSSIBLE ERROR:
Nobeli Nobeli O PROPN
POSSIBLE ERROR:
Tokyo-kodus Tokyo-kodus O PROPN
POSSIBLE ERROR:
Nobelist Nobelist O PROPN
POSSIBLE ERROR:
Bensiini Bensiini O PROPN
POSSIBLE ERROR:
Lääne Lääne O PROPN
POSSIBLE ERROR:
Kindlustus Kindlustus O PROPN
POSSIBLE ERROR:
Väärtpaberibörsi Väärtpaberibörsi O PROPN
POSSIBLE ERROR:
Dollar Dollar O PROPN
POSSIBLE ERROR:
Korterivõtmete Korterivõtmete O PROPN
POSSIBLE ERROR:
Ühispank Ühispank O PROPN
POSSIBLE ERROR:
Terve Terve O PROPN
POSSIBLE ERROR:
Ühispanga Ühispanga O PROPN
POSSIBLE ERROR:
Atleedi Atleedi O PROPN
POSSIBLE ERROR:
Kanada Kanada O PROPN
POSSIBLE ERROR:
GP G

In [89]:
possible_errors

1565

In [61]:
len(correction_dict)

73

In [64]:
with open('data/estner.json') as corpus:
    text_idx = 0
    for doc in corpus:
        if correction_dict[text_idx]:
            estnltk_dict = json.loads(doc.strip())
            word_dict = estnltk_dict.get('words', [])      
            for idx, word in enumerate(word_dict):
                if idx in correction_dict[text_idx]:
                    print('Document idx:', text_idx)
                    print('Word idx:', idx)
                    print(with_context(idx, word_dict))
        text_idx += 1

Document idx: 2
Word idx: 88
['kaitsejõudude', 'leitnant', 'Tiit', 'Uustalu', '.', '”', 'Uustalu', 'väitel', 'veeti']
Document idx: 10
Word idx: 39
['Zimbabwesse', ',', 'teatab', 'CNN', '.', 'Uus-Meremaa', 'välisministri', 'sõnul', 'on']
Document idx: 13
Word idx: 87
['asjaolude', 'uurimiseks', 'on', 'majandus', '-', 'ja', 'kommunikatsiooniministeeriumis', 'moodustatud', 'komisjon']
Document idx: 14
Word idx: 36
['bussile', 'eelistavad', '.', 'Teede', '-', 'ja', 'sideministeeriumi', 'pressiesindaja', 'Kuldar']
Document idx: 22
Word idx: 71
['sõnas', 'Kukumägi', 'ajakirjas', 'Teater.Muusika.Kino', '.', 'viimases', 'numbris', '.', 'Samas']
Document idx: 36
Word idx: 84
['aprillikuu', 'seisuga', 'on', 'maksu', '-', 'ja', 'tolliameti', 'andmetel', 'väljastatud']
Document idx: 42
Word idx: 127
['ilmselt', 'alles', 'maandumisel', 'Tallinnas', '.', '”', 'Lennuki', 'teliku', 'rehvid']
Document idx: 42
Word idx: 159
['võimalikust', 'ohuolukorrast', 'Tallinna', 'tuletõrje', '-', 'ja', 'päästeame

Document idx: 459
Word idx: 49
['Valdmann', 'ning', 'Tallinna', 'tuletõrje', '-', 'ja', 'päästeameti', 'juht', 'Raik']
Document idx: 459
Word idx: 182
['summa', 'sai', 'Tallinna', 'tuletõrje', '-', 'ja', 'päästeameti', 'juht', 'Raik']
Document idx: 472
Word idx: 39
[',', 'teatas', 'erakond', 'Delfile', '.', '2004.', 'aastal', 'annetasid', 'eraisikud']
Document idx: 484
Word idx: 7147
['Eesti', 'meest', '-', 'Telia', '/', 'Starmani', 'juht', 'Peeter', 'Kern']
Document idx: 484
Word idx: 8994
['Liit', 'Eesti', 'Puutööliit', 'Metsamajandusliit', '“', 'Eesti', 'Mets', '”', 'Eesti']
Document idx: 484
Word idx: 8997
['Metsamajandusliit', '“', 'Eesti', 'Mets', '”', 'Eesti', 'Paberitööstuse', 'Liit', 'Eesti']
Document idx: 484
Word idx: 10920
['Hiljuti', 'sain', 'lepinguliselt', 'SE', '&', 'JS', 'kirjastuse', 'toimetajaks', '.']
Document idx: 484
Word idx: 19074
['väeosad', 'ründasid', 'valitsusarmeed', 'Ponnalaikadduwani', ',', 'Navakkeri', 'ja', 'Puthur', 'Westi']
Document idx: 485
Word idx:

In [66]:
punct_corrections = {
2: [88],
10: [39],
42: [127],
47: [140],
53: [77],
63: [54, 215],
75: [30],
83: [310],
92: [43],
162: [34],
163: [48],
186: [69],
205: [51],
246: [69],
248: [65],
249: [63],
265: [59],
269: [84],
293: [184],
314: [21],
328: [97],
356: [128],
414: [127],
416: [134, 202],
420: [47, 122],
423: [19],
426: [140],
437: [94],
472: [39],
487: [278, 280],
518: [22],
541: [649]
}

In [71]:
changes_made = 0
with open('data/estner.json') as corpus:
    with open('data/estner_punct_cleaned.json', 'w') as dest:
        text_idx = 0
        for doc in corpus:
            estnltk_dict = json.loads(doc.strip())
            if text_idx in punct_corrections:
                word_dict = estnltk_dict.get('words', [])
                new_words = []
                for idx, word in enumerate(word_dict):
                    if idx in punct_corrections[text_idx]:
                        # Make sure we don't delete a tag followed by
                        # an I-tag
                        if idx < len(word_dict)-1:
                            if not word_dict[idx+1]['label'].startswith('I'):
                                word['label'] = 'O'
                                changes_made += 1
                        else:
                            word['label'] = 'O'
                            changes_made += 1
                    new_words.append(word)
                estnltk_dict['words'] = new_words
            json.dump(estnltk_dict, dest)
            dest.write('\n')
            text_idx += 1

In [72]:
changes_made

32

In [73]:
label_counts = Counter()
total_words = 0
doc_count = 0
doc_lengths = []

with open('data/estner_punct_cleaned.json') as estnltk_json:
    for line in estnltk_json:
        estnltk_dict = json.loads(line.strip())
        doc_count += 1
        doc_length = 0
        for word in estnltk_dict.get('words', []):
            total_words += 1
            label_counts[word['label']] += 1
            doc_length += 1
        doc_lengths.append(doc_length)

In [74]:
label_counts

Counter({'B-LOC': 5711,
         'O': 197064,
         'B-ORG': 3938,
         'I-ORG': 2313,
         'B-PER': 5762,
         'I-PER': 2883,
         'I-LOC': 654})

In [75]:
total_words

218325

In [87]:
word_count = 0
with open('data/estner.cnll', encoding='utf-8') as corpus:
    with open('data/estner.json') as corpus_2:
        doc_idx = 0
        for line in corpus_2:
            corpus_2_dict = json.loads(line.strip())
            corpus_2_words = corpus_2_dict['words']

            for word in corpus_2_words:
                corpus_line = corpus.readline().strip()
                if not corpus_line:
                    corpus_line = corpus.readline().strip()
                print(word['text'], corpus_line.split()[0])
            doc_idx += 1

Tallinna Tallinna
õhusaaste õhusaaste
suureneb suureneb
. .
Kuigi Kuigi
möödunud möödunud
nädala nädala
ühel ühel
ööl ööl
ootamatult ootamatult
kõrgeks kõrgeks
tõusnud tõusnud
õhusaaste õhusaaste
oli oli
pealinna pealinna
jaoks jaoks
haruldane haruldane
rekord rekord
, ,
liigub liigub
linnaõhu linnaõhu
saastekõver saastekõver
järjekindlalt järjekindlalt
ülespoole ülespoole
, ,
konkureerides konkureerides
teiste teiste
Euroopa Euroopa
pealinnadega pealinnadega
. .
Nagu Nagu
kirjutab kirjutab
Postimees Postimees
, ,
registreeriti registreeriti
Taksopargi Taksopargi
õhuseirejaamas õhuseirejaamas
11. 11.märtsil
märtsil Tallinna
Tallinna õhusaastumise
õhusaastumise rekord
rekord .
. Kuigi
Kuigi kümme
kümme aastat
aastat tagasi
tagasi oli
oli pealinna
pealinna õhk
õhk praegusest
praegusest märksa
märksa mustem
mustem ,
, ei
ei vabanda
vabanda see
see linna
linna keskkonnaspetsialistide
keskkonnaspetsialistide sõnul
sõnul linnaõhu
linnaõhu taas
taas halvemaks
halvemaks muutumist
muutumist .
.

Lechil nina
on peal
põse kaks
ja täppi
nina .
peal --
kaks Vilnius
täppi tahab
. CNN-i
Vilnius ilmateatesse
tahab saada
CNN-i .
ilmateatesse Vilniuse
saada energiline
. ja
Vilniuse ambitsioonikas
energiline linnapea
ja ,
ambitsioonikas liberaal
linnapea Arturas
, Zuokas
liberaal soovib
Arturas ,
Zuokas et
soovib Vilniuse
, ilmateadet
et edastataks
Vilniuse ka
ilmateadet rahvusvahelistes
edastataks telekanalites
ka .
rahvusvahelistes Zuokas
telekanalites ja
. tema
Zuokas nõunikud
ja peavad
tema praegu
nõunikud läbirääkimisi
peavad telekanalitega
praegu Cable
läbirääkimisi News
telekanalitega Network
Cable (
News CNN
Network )
( ja
CNN Independent
) Television
ja News
Independent (
Television ITN
News )
( ,
ITN et
) Euroopa
, ilmakaardile
et nende
Euroopa saadetes
ilmakaardile ilmuks
nende ka
saadetes Vilnius
ilmuks .
ka Kesk
Vilnius -
. ja
Kesk Ida-Euroopa
- kaartidel
ja ,
Ida-Euroopa mida
kaartidel need
, kanalid
mida näitavad
need on
kanalid märgitud
näitavad ainult
on Minsk
märgitud 

lisaks õnnetust
on juhtus
reisijatel binokliga
päästevestid nägema
. ka
Päästeameti üks
operatiivkorrapidaja eraisik
Alo ,
Tammsalu kes
ütles samuti
Äripäev teatas
Online õnnetusest
’ .
ile Naissaarel
, ametis
et olev
õnnetust mees
juhtus ütles
binokliga Äripäev
nägema Online’ile
ka ,
üks et
eraisik sealse
, piirivalve
kes teatel
samuti oli
teatas kopter
õnnetusest kukkunud
. otse
Naissaarel merre
ametis ,
olev kuid
mees saarelt
ütles seda
Äripäev paista
Online pole
’ .
ile Mehe
, sõnul
et kukkus
sealse kopter
piirivalve merre
teatel Tallinn-2
oli poi
kopter juures
kukkunud ,
otse teispool
merre Naissaart
, .
kuid Kopteriõnnetuse
saarelt asjus
seda algatatakse
paista kriminaalmenetlus
pole ,
. öeldi
Mehe Delfile
sõnul riigiprokuratuurist
kukkus .
kopter Menetlus
merre alustati
Tallinn-2 liiklusohutusnõuete
poi ja
juures õhusõiduki
, käitusnõuete
teispool rikkumise
Naissaart paragrahvi
. alusel
Kopteriõnnetuse .
asjus Tõnis
algatatakse Lepp
kriminaalmenetlus ütles
, “
öeldi Aktuaalsele


ja kuidagi
sellega nõus
, olla
et ministri
riigikogule süüdistustega
tehakse parlamendi
etteheiteid aadressil
tema ,
otsuste kes
pärast ei
, andnud
” ligi
lausus miljardit
Ligi krooni
Eesti Edelaraudtee
Päevalehe dotatsiooniks
teatel .
. Jürgensoni
Tema jätkamist
sõnul ametis
ei toetas
saa siiski
kuidagi kogu
nõus koalitsioon
olla .
ministri Tema
süüdistustega umbusaldamise
parlamendi poolt
aadressil hääletas
, 42
kes opositsioonisaadikut
ei ,
andnud kes
ligi süüdistasid
miljardit ministrit
krooni peamiselt
Edelaraudtee raudteel
dotatsiooniks reisijateveo
. väljasuretamises
Jürgensoni ja
jätkamist Kagu-Eesti
ametis inimeste
toetas liikumisvõimaluste
siiski piiramises
kogu .
koalitsioon Rünnakuid
. tõrjunud
Tema Jürgenson
umbusaldamise kinnitas
poolt ,
hääletas et
42 1.
opositsioonisaadikut märtsiks
, ,
kes kui
süüdistasid lõpeb
ministrit leping
peamiselt Edelaraudteega
raudteel ,
reisijateveo peab
väljasuretamises riik
ja suutma
Kagu-Eesti kõik
inimeste probleemid
liikumisvõimaluste bu

Sellised Näitusekülastajad
me saavad
oleme tutvuda
. 22
New täiesti
Yorgis terve
on inimkehaga
avatud ning
näitus lisaks
inimorganismist sellele
, on
kus eraldi
on väljas
väljas 260
kümned haigusest
inimkeha puretut
mudelid organit
. .
Näitusekülastajad Seejuures
saavad saab
tutvuda võrrelda
22 üht
täiesti ja
terve sama
inimkehaga elundit
ning eri
lisaks haigusestaadiumis
sellele .
on --
eraldi Kukumägi
väljas ähvardas
260 Remsul
haigusest jalad
puretut murda
organit .
. Näitleja
Seejuures Arvo
saab Kukumägi
võrrelda vihjab
üht võimalusele
ja ,
sama et
elundit ta
eri võib
haigusestaadiumis kirjamees
. Olev
Kukumägi Remsu
ähvardas jalaluud
Remsul ära
jalad murda
murda .
. Nimelt
Näitleja ei
Arvo istu
Kukumägi näitlejale
vihjab ,
võimalusele et
, kirjamees
et andis
ta temast
võib kui
kirjamees alkohoolikust
Olev aastate
Remsu eest
jalaluud tehtud
ära filmi
murda „
. Uuesti
Nimelt elus
ei ”
istu ETV-le
näitlejale näidata
, .
et ”
kirjamees Kui
andis ühel
temast päeval
kui Remsul
alkohooli

...

Küll talle
aga nii
võttis isiklikud
politsei ihukaitsjad
paari kui
nädala ka
eest mehitatud
täiemahulise saatemaasturi
kaitse .
alla Seni
riigikogu oli
esimehe tähtsuselt
, teine
andes inimene
talle riigis
nii ihukaitsest
isiklikud loobunud
ihukaitsjad ,
kui nii
ka nagu
mehitatud oli
saatemaasturi otsustanud
. ka
Seni tema
oli eelkäija
tähtsuselt Toomas
teine Savi
inimene .
riigis Uute
ihukaitsest ülesannete
loobunud tõttu
, vajab
nii julgestuspolitsei
nagu aastas
oli veel
otsustanud kümmet
ka miljonit
tema krooni
eelkäija ,
Toomas täiendamaks
Savi oma
. veidi
Uute enam
ülesannete kui
tõttu 50
vajab miljoni
julgestuspolitsei kroonist
aastas eelarvet
veel .
kümmet --
miljonit Kesk
krooni nõus
, teatud
täiendamaks tingimustel
oma Rüütlit
veidi toetama
enam .
kui Keskerakond
50 teatas
miljoni eile
kroonist ,
eelarvet et
. ootab
Kesk Arnold
nõus Rüütlilt
teatud selget
tingimustel seisukohta
Rüütlit presidendiks
toetama kandideerimise
. kohta
Keskerakond ning
teatas on
eile valmis
, teda
et

on linna
Eestis elurajoonides
ligi ,
25 kus
000 vilkuvad
hasartmängusõltlast tuled
, häirivad
mis sealseid
on elanikke
2,4 .
% Hasartmängukohtade
elanikkonnast välisvalgustuses
. kutsuvalt
Hasartmängukohad vilkuvad
paiknevad tuled
tihti on
linna ahvatluseks
elurajoonides just
, noortele
kus ,
vilkuvad kelle
tuled väärtushinnangud
häirivad pole
sealseid veel
elanikke välja
. kujunenud
Hasartmängukohtade ,
välisvalgustuses kes
kutsuvalt on
vilkuvad kergesti
tuled mõjutatavad
on ning
ahvatluseks kellel
just hasartmängusõltuvus
noortele kujuneb
, välja
kelle kergemini
väärtushinnangud .
pole Vilkuvate
veel reklaamtulede
välja kasutamise
kujunenud keelamine
, hasartmängukohtade
kes välisvalgustuses
on on
kergesti vahend
mõjutatavad ,
ning mis
kellel kaotab
hasartmängusõltuvus öisest
kujuneb linnapildist
välja elanikke
kergemini häirivad
. tuled
Vilkuvate ning
reklaamtulede loodetavasti
kasutamise muudab
keelamine mängukohtade
hasartmängukohtade asukohad
välisvalgustuses noortele
on vähematr

NSV tonni
Liidus muud
valmistatud laadungit
üks .
suuremaid TU-154
reisilennukeid võib
, ilma
mis vahemaandumiseta
võib lennata
pardale kuni
võtta 7000
166 kilomeetrit
reisijat .
ja --
4,56 Soomes
tonni uppus
muud Eesti
laadungit kalamees
. .
TU-154 Soomes
võib Savonlinnas
ilma leiti
vahemaandumiseta laupäeva
lennata hilisõhtul
kuni uppununa
7000 66-aastane
kilomeetrit kalastamas
. olnud
Soomes Eesti
uppus mees
Eesti .
kalamees Agentuuri
. STT
Soomes teatel
Savonlinnas läks
leiti mees
laupäeva varahommikul
hilisõhtul sõudepaadiga
uppununa Ahvensalmi
66-aastane lähistel
kalastamas järvele
olnud .
Eesti Veidi
mees enne
. keskpäeva
Agentuuri nähti
STT paati
teatel tühjalt
läks triivimas
mees ,
varahommikul vahendab
sõudepaadiga PM
Ahvensalmi Online
lähistel BNS-i
järvele uudist
. .
Veidi Sukelduja
enne leidis
keskpäeva mehe
nähti seitsme
paati meetri
tühjalt sügavuselt
triivimas .
, Savonlinna
vahendab politsei
PM peab
Online tõenäoliseks
BNS-i ,
uudist et
. mees
Sukelduja takerdus
leidis

Eelmodernne Seepärast
ühiskond on
on Laaril
agraarühiskond Kolga
, arvates
kus vaja
tähtsad 1,5
on miljonit
maa eestikeelset
ja inimest
rahva .
hulk “
. Kuna
” maa
Seepärast on
on esmane
Laaril väärtus
Kolga ,
arvates on
vaja ka
1,5 piirid
miljonit pühad
eestikeelset .
inimest Sümbolitel
. on
“ üldse
Kuna suur
maa roll
on :
esmane nende
väärtus pihta
, on
on mõnus
ka tulistada
piirid ,
pühad ”
. kiikab
Sümbolitel Kolga
on Laari
üldse pähe
suur .
roll Ei
: maksa
nende unustada
pihta ka
on seda
mõnus ,
tulistada et
, kolhoosid
” suurtootmise
kiikab ühe
Kolga variandina
Laari eelmodernsesse
pähe maailma
. ei
Ei sobi
maksa .
unustada Ansipi
ka ja
seda tema
, modernsuse
et loogika
kolhoosid on
suurtootmise teistsugune
ühe .
variandina Piiridel
eelmodernsesse ,
maailma maal
ei ja
sobi rahvusel
. ei
Ansipi ole
ja tähtsust
tema ,
modernsuse olulised
loogika on
on tööjõud
teistsugune ja
. kapital
Piiridel .
, Modernne
maal on
ja ratsionaalne
rahvusel ,
ei kirjeldab
ole professor
tähtsust modern

KeyboardInterrupt: 

In [82]:
word_count

217824

Võrreldes estner.json korpust (saadud keeleressursside gitist, estnltk gitis enam üleval ei ole, aga on näha, et json-formaadis korpuse peal on NER-mudel treenitud) ja esialgses artiklis viidatus estner.cnll korpust, on näha, et vahepeal (2013 vs viimati uuendatud 2015) on veidi parandatud tokeniseerimist, mistõttu ka erinev koguarv, aga sama nimeüksuste arv.

In [134]:
# Create 116 (58+58) random ints in the range of the number
# of documents, use them to split the original corpus into 
# train, dev, test splits
idxs = np.random.choice(572, size=116, replace=False)

In [135]:
dev_idxs = np.random.choice(idxs, size=58, replace=False)

In [136]:
# Split the corpus into train and test sets
with open('data/estner_punct_cleaned.json') as estnltk_json:
    with open('data/project_train.json', 'w') as train:
        with open('data/project_dev.json', 'w') as dev:
            with open('data/project_test.json', 'w') as test:
                i = 0
                train_count = 0
                dev_count = 0
                test_count = 0
                for line in estnltk_json:
                    
                    estnltk_dict = json.loads(line.strip())
                    if i in dev_idxs:
                        json.dump(estnltk_dict, dev)
                        dev.write('\n')
                        dev_count += 1
                    elif i in idxs:
                        json.dump(estnltk_dict, test)
                        test.write('\n')
                        test_count += 1
                    else:
                        json.dump(estnltk_dict, train)
                        train.write('\n')
                        train_count += 1
                    i += 1

In [137]:
print(train_count, dev_count, test_count)

456 58 58


In [17]:
data_files = ['project_train', 'project_dev',
             'project_test']

for name in data_files:
    label_counts = Counter()
    total_words = 0

    with open('data/project_estner/{0}.json'.format(name)) as file:
        for line in file:
            estnltk_dict = json.loads(line.strip())
            for word in estnltk_dict.get('words', []):
                total_words += 1
                label_counts[word['label']] += 1

    print(name)
    print(label_counts)
    print(total_words, '\n')
    for label in label_counts:
        proportion = round(label_counts[label] / total_words * 100, 2)
        print('Proportion of label {0}: {1}'.format(label, proportion))
    print('\n')

project_train
Counter({'O': 151143, 'B-PER': 4531, 'B-LOC': 4418, 'B-ORG': 3064, 'I-PER': 2238, 'I-ORG': 1833, 'I-LOC': 472})
167699 

Proportion of label B-LOC: 2.63
Proportion of label O: 90.13
Proportion of label B-ORG: 1.83
Proportion of label I-ORG: 1.09
Proportion of label B-PER: 2.7
Proportion of label I-PER: 1.33
Proportion of label I-LOC: 0.28


project_dev
Counter({'O': 17346, 'B-LOC': 490, 'B-PER': 354, 'B-ORG': 274, 'I-PER': 195, 'I-ORG': 175, 'I-LOC': 64})
18898 

Proportion of label B-LOC: 2.59
Proportion of label O: 91.79
Proportion of label B-ORG: 1.45
Proportion of label I-ORG: 0.93
Proportion of label B-PER: 1.87
Proportion of label I-PER: 1.03
Proportion of label I-LOC: 0.34


project_test
Counter({'O': 28575, 'B-PER': 877, 'B-LOC': 803, 'B-ORG': 600, 'I-PER': 450, 'I-ORG': 305, 'I-LOC': 118})
31728 

Proportion of label O: 90.06
Proportion of label B-ORG: 1.89
Proportion of label I-ORG: 0.96
Proportion of label B-LOC: 2.53
Proportion of label B-PER: 2.76
Proportion 

In [140]:
with open('data/project_test.json') as corpus:
    doc_idx = 0
    for doc in corpus:
        print(doc_idx)
        estnltk_dict = json.loads(doc.strip())
        pprint(estnltk_dict)
        doc_idx += 1

0
{'paragraphs': [{'end': 1396, 'start': 0}],
 'sentences': [{'end': 37, 'start': 0},
               {'end': 210, 'start': 38},
               {'end': 316, 'start': 211},
               {'end': 439, 'start': 317},
               {'end': 578, 'start': 440},
               {'end': 632, 'start': 579},
               {'end': 741, 'start': 633},
               {'end': 871, 'start': 742},
               {'end': 1092, 'start': 872},
               {'end': 1142, 'start': 1093},
               {'end': 1204, 'start': 1143},
               {'end': 1358, 'start': 1205},
               {'end': 1396, 'start': 1359}],
 'text': 'Rästikud häirivad koerte treenimist .\n'
         'Eesti koeraspordiliidu Harku karjääri lähedal paikneva '
         'treeningväljaku kasutamine on kevade tulekuga maa seest välja '
         'pugenud rästikute tõttu sisuliselt võimatuks muutunud .\n'
         '“ Nädalavahetusel toimus meil seal koerakutsikate näitus ja platsile '
         'viiv asfalttee kubises rästikutest .\

           {'end': 337, 'label': 'O', 'start': 333, 'text': 'seda'},
           {'end': 343, 'label': 'O', 'start': 338, 'text': 'keeta'},
           {'end': 345, 'label': 'O', 'start': 344, 'text': ','},
           {'end': 354, 'label': 'O', 'start': 346, 'text': 'vahendab'},
           {'end': 358, 'label': 'B-ORG', 'start': 355, 'text': 'EPL'},
           {'end': 365, 'label': 'I-ORG', 'start': 359, 'text': 'Online'},
           {'end': 367, 'label': 'O', 'start': 366, 'text': '.'},
           {'end': 376, 'label': 'O', 'start': 368, 'text': 'Joogivee'},
           {'end': 394,
            'label': 'O',
            'start': 377,
            'text': 'mikrobioloogiline'},
           {'end': 405, 'label': 'O', 'start': 395, 'text': 'saastumine'},
           {'end': 410, 'label': 'O', 'start': 406, 'text': 'võib'},
           {'end': 421, 'label': 'O', 'start': 411, 'text': 'põhjustada'},
           {'end': 441,
            'label': 'O',
            'start': 422,
            'text': 'ep

           {'end': 187, 'label': 'O', 'start': 176, 'text': 'vabastamine'},
           {'end': 194, 'label': 'O', 'start': 188, 'text': 'toimus'},
           {'end': 202, 'label': 'O', 'start': 195, 'text': 'avaliku'},
           {'end': 213, 'label': 'O', 'start': 203, 'text': 'teenistuse'},
           {'end': 221, 'label': 'O', 'start': 214, 'text': 'seaduse'},
           {'end': 228, 'label': 'O', 'start': 222, 'text': 'alusel'},
           {'end': 235, 'label': 'O', 'start': 229, 'text': 'seoses'},
           {'end': 240, 'label': 'B-PER', 'start': 236, 'text': 'Jüri'},
           {'end': 247, 'label': 'I-PER', 'start': 241, 'text': 'Ratase'},
           {'end': 258, 'label': 'O', 'start': 248, 'text': 'valimisega'},
           {'end': 268, 'label': 'B-ORG', 'start': 259, 'text': 'Riigikogu'},
           {'end': 277, 'label': 'O', 'start': 269, 'text': 'liikmeks'},
           {'end': 279, 'label': 'O', 'start': 278, 'text': '.'},
           {'end': 288, 'label': 'O', 'start': 280, 

           {'end': 151, 'label': 'O', 'start': 141, 'text': 'teenindama'},
           {'end': 158, 'label': 'O', 'start': 152, 'text': 'bussid'},
           {'end': 160, 'label': 'O', 'start': 159, 'text': ','},
           {'end': 166, 'label': 'O', 'start': 161, 'text': 'mille'},
           {'end': 175, 'label': 'O', 'start': 167, 'text': 'keskmine'},
           {'end': 181, 'label': 'O', 'start': 176, 'text': 'vanus'},
           {'end': 184, 'label': 'O', 'start': 182, 'text': 'ei'},
           {'end': 190, 'label': 'O', 'start': 185, 'text': 'ületa'},
           {'end': 193, 'label': 'O', 'start': 191, 'text': '13'},
           {'end': 200, 'label': 'O', 'start': 194, 'text': 'aastat'},
           {'end': 202, 'label': 'O', 'start': 201, 'text': ','},
           {'end': 208, 'label': 'O', 'start': 203, 'text': 'ütles'},
           {'end': 230,
            'label': 'O',
            'start': 209,
            'text': 'majandusministeeriumi'},
           {'end': 246, 'label': 'O', 'sta

           {'end': 411, 'label': 'O', 'start': 410, 'text': ','},
           {'end': 420, 'label': 'O', 'start': 412, 'text': 'kirjutab'},
           {'end': 430, 'label': 'B-ORG', 'start': 421, 'text': 'Postimees'},
           {'end': 432, 'label': 'O', 'start': 431, 'text': '.'},
           {'end': 446, 'label': 'O', 'start': 433, 'text': 'Maavalitsuste'},
           {'end': 454, 'label': 'O', 'start': 447, 'text': 'andmete'},
           {'end': 461, 'label': 'O', 'start': 455, 'text': 'põhjal'},
           {'end': 464, 'label': 'O', 'start': 462, 'text': 'on'},
           {'end': 471, 'label': 'O', 'start': 465, 'text': 'pandud'},
           {'end': 477, 'label': 'O', 'start': 472, 'text': 'kokku'},
           {'end': 486, 'label': 'O', 'start': 478, 'text': 'nimekiri'},
           {'end': 488, 'label': 'O', 'start': 487, 'text': ','},
           {'end': 492, 'label': 'O', 'start': 489, 'text': 'mis'},
           {'end': 501, 'label': 'O', 'start': 493, 'text': 'sisaldab'},
        

...

            'label': 'O',
            'start': 10828,
            'text': 'ülikoolimaastik'},
           {'end': 10850, 'label': 'O', 'start': 10844, 'text': 'hakkab'},
           {'end': 10859, 'label': 'O', 'start': 10851, 'text': 'lähiajal'},
           {'end': 10872, 'label': 'O', 'start': 10860, 'text': 'paratamatult'},
           {'end': 10888,
            'label': 'O',
            'start': 10873,
            'text': 'diferentseeruma'},
           {'end': 10890, 'label': 'O', 'start': 10889, 'text': ','},
           {'end': 10895, 'label': 'O', 'start': 10891, 'text': 'ning'},
           {'end': 10900, 'label': 'O', 'start': 10896, 'text': 'seda'},
           {'end': 10909, 'label': 'O', 'start': 10901, 'text': 'eelkõige'},
           {'end': 10919, 'label': 'O', 'start': 10910, 'text': 'teadustöö'},
           {'end': 10922, 'label': 'O', 'start': 10920, 'text': 'ja'},
           {'end': 10933, 'label': 'O', 'start': 10923, 'text': 'kraadiõppe'},
           {'end': 10940, 'label

           {'end': 13455, 'label': 'I-LOC', 'start': 13450, 'text': 'riigi'},
           {'end': 13466, 'label': 'O', 'start': 13456, 'text': 'tulevikule'},
           {'end': 13468, 'label': 'O', 'start': 13467, 'text': '.'},
           {'end': 13476, 'label': 'O', 'start': 13469, 'text': 'Sellise'},
           {'end': 13482, 'label': 'O', 'start': 13477, 'text': 'karmi'},
           {'end': 13492, 'label': 'O', 'start': 13483, 'text': 'järelduse'},
           {'end': 13497, 'label': 'O', 'start': 13493, 'text': 'saab'},
           {'end': 13502, 'label': 'O', 'start': 13498, 'text': 'teha'},
           {'end': 13511, 'label': 'O', 'start': 13503, 'text': 'otseselt'},
           {'end': 13519, 'label': 'O', 'start': 13512, 'text': 'nendest'},
           {'end': 13534,
            'label': 'O',
            'start': 13520,
            'text': 'ettepanekutest'},
           {'end': 13536, 'label': 'O', 'start': 13535, 'text': ','},
           {'end': 13540, 'label': 'O', 'start': 13537, '

            'text': 'opositsioonilt'},
           {'end': 16579,
            'label': 'O',
            'start': 16564,
            'text': 'konstruktiivset'},
           {'end': 16589, 'label': 'O', 'start': 16580, 'text': 'käitumist'},
           {'end': 16591, 'label': 'O', 'start': 16590, 'text': '.'},
           {'end': 16601, 'label': 'O', 'start': 16592, 'text': 'Küüniline'},
           {'end': 16604, 'label': 'O', 'start': 16602, 'text': 'on'},
           {'end': 16607, 'label': 'O', 'start': 16605, 'text': 'ka'},
           {'end': 16620,
            'label': 'B-ORG',
            'start': 16608,
            'text': 'Keskerakonna'},
           {'end': 16630, 'label': 'O', 'start': 16621, 'text': 'ettepanek'},
           {'end': 16636, 'label': 'O', 'start': 16631, 'text': 'kanda'},
           {'end': 16643, 'label': 'O', 'start': 16637, 'text': 'tulude'},
           {'end': 16651, 'label': 'O', 'start': 16644, 'text': 'poolele'},
           {'end': 16655, 'label': 'O', 'start': 

           {'end': 18896, 'label': 'O', 'start': 18895, 'text': '-'},
           {'end': 18903, 'label': 'O', 'start': 18897, 'text': 'erinev'},
           {'end': 18911, 'label': 'O', 'start': 18904, 'text': 'nägemus'},
           {'end': 18917, 'label': 'O', 'start': 18912, 'text': '1999.'},
           {'end': 18923, 'label': 'O', 'start': 18918, 'text': 'aasta'},
           {'end': 18930, 'label': 'O', 'start': 18924, 'text': 'tulude'},
           {'end': 18940, 'label': 'O', 'start': 18931, 'text': 'laekumise'},
           {'end': 18951, 'label': 'O', 'start': 18941, 'text': 'prognoosis'},
           {'end': 18953, 'label': 'O', 'start': 18952, 'text': ';'},
           {'end': 18955, 'label': 'O', 'start': 18954, 'text': '-'},
           {'end': 18972,
            'label': 'O',
            'start': 18956,
            'text': 'venitamistaktika'},
           {'end': 18979, 'label': 'O', 'start': 18973, 'text': 'eelnõu'},
           {'end': 18996,
            'label': 'O',
           

           {'end': 21291,
            'label': 'O',
            'start': 21278,
            'text': 'eelarvetulude'},
           {'end': 21302, 'label': 'O', 'start': 21292, 'text': 'puudujäägi'},
           {'end': 21310, 'label': 'O', 'start': 21303, 'text': 'katteks'},
           {'end': 21314, 'label': 'O', 'start': 21311, 'text': 'või'},
           {'end': 21320, 'label': 'O', 'start': 21315, 'text': 'hoida'},
           {'end': 21324, 'label': 'O', 'start': 21321, 'text': 'see'},
           {'end': 21332, 'label': 'O', 'start': 21325, 'text': 'näiteks'},
           {'end': 21352,
            'label': 'O',
            'start': 21333,
            'text': 'pensionikindlustuse'},
           {'end': 21360, 'label': 'O', 'start': 21353, 'text': 'reformi'},
           {'end': 21374,
            'label': 'O',
            'start': 21361,
            'text': 'läbiviimiseks'},
           {'end': 21376, 'label': 'O', 'start': 21375, 'text': '?'},
           {'end': 21381, 'label': 'O', 'star

In [13]:
punctuation = re.compile('\.|\,$')
with open('data/estner.cnll', encoding='utf-8') as corpus:
    for line in corpus:
        line = line.strip()
        if line and line != '--':
            text = line.split()[0]
            label = line.split()[-1]

            if label != 'O' and (re.match(punctuation, text) or
                                 text.endswith(('.', ','))):
                print('ERROR:', text, 'labelled as', label)

ERROR: Uustalu. labelled as I-PER
ERROR: CNN. labelled as B-ORG
ERROR: Teater.Muusika.Kino. labelled as B-ORG
ERROR: W. labelled as I-PER
ERROR: Tallinnas. labelled as B-LOC
ERROR: Jõgiste. labelled as I-PER
ERROR: Petti. labelled as I-PER
ERROR: BBC. labelled as B-ORG
ERROR: Venemaa. labelled as B-LOC
ERROR: USA. labelled as B-LOC
ERROR: W. labelled as I-PER
ERROR: Bagliani. labelled as B-PER
ERROR: BBC. labelled as B-ORG
ERROR: Annabel/Annabell. labelled as B-LOC
ERROR: CNN. labelled as B-ORG
ERROR: ringkonnaprokuratuur. labelled as I-ORG
ERROR: PHARE. labelled as B-ORG
ERROR: W. labelled as I-PER
ERROR: päästeamet. labelled as B-ORG
ERROR: BBC. labelled as B-ORG
ERROR: D.W. labelled as B-PER
ERROR: . labelled as B-ORG
ERROR: NATO. labelled as B-ORG
ERROR: Leht. labelled as I-ORG
ERROR: AFP. labelled as B-ORG
ERROR: Jaan. labelled as B-PER
ERROR: AS. labelled as I-ORG
ERROR: Ekspress. labelled as I-ORG
ERROR: W. labelled as I-PER
ERROR: W. labelled as I-PER
ERROR: W. labelled as I-PE

In [14]:
data_files = ['project_train', 'project_dev', 'project_test']

label_counts = Counter()
total_words = 0
for name in data_files:
    with open('data/project_estner/{0}.json'.format(name), encoding='utf-8') as file:  
        sentences = []
        with open('data/project_estner/{0}.txt'.format(name), 'w', encoding='utf-8') as out:
            for line in file:
                estnltk_dict = json.loads(line.strip())
                estnltk_sentences = estnltk_text_to_sentences(estnltk_dict)
                sentences.extend(estnltk_sentences)
            for sentence in sentences:
                for token_pair in sentence:
                    label_counts[token_pair[1]] += 1
                    out.write('{0}\t{1}\n'.format(token_pair[0], token_pair[1]))
                out.write('\n')

In [15]:
label_counts

Counter({'B-LOC': 5711,
         'O': 197064,
         'B-ORG': 3938,
         'I-ORG': 2313,
         'B-PER': 5762,
         'I-PER': 2883,
         'I-LOC': 654})